In [96]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
import load_data
import nan_imputation
import helpers
from helpers import find_repo_root
from sksurv.nonparametric import kaplan_meier_estimator

In [ ]:
import importlib
importlib.reload(load_data)
importlib.reload(nan_imputation)
importlib.reload(helpers)

## Step 0 : Load the Data, we load all Lifespan folder expect of Terebafin
> We don't load terebafin, we can see and modify this in the code of Load_data

In [98]:
repo_root = find_repo_root()
repo_root

data_path = os.path.join(repo_root, 'Data/Lifespan')

In [99]:
worms = load_data.load_lifespan(data_path)

In [ ]:
# just a check print on worm 3 (companyDrug)
worm_name = 'worm_3'  # Change this to the name of the worm you want to print
print(f"Worm: {worm_name}")
worm_data = worms[worm_name]
df = pd.DataFrame(worm_data.T, columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category'])
print(df)

In [ ]:
helpers.print_fdict_summary(worms)  

## Step 1 : NaN imputation
> impute only on X and Y columns since only where there are NaN

In [ ]:
for name, lifespan_array in worms.items(): 
    print(f"Processing {name}")
    lifespan_arrayxy = lifespan_array[2:4,:]  # Extract columns for X and Y
    missing_sequences = nan_imputation.count_successive_missing(lifespan_arrayxy)
    for start, end, length in missing_sequences:
        print(f"  Missing sequence starts at column {start}, ends at column {end - 1}, length: {length}")

In [ ]:
#print(lifespan_arrayxy)
print(f"Missing sequences for {name}: {missing_sequences}")

In [105]:
# Rows to check for missing values (2:4 in zero-based indexing)
rows_to_check = slice(2, 4)  # Rows 2 and 3 not row 4

# Apply cut_array to each worm in the dataset
cut_nan_dict = {name: nan_imputation.cut_array(array, rows_to_check) for name, array in worms.items()}

In [ ]:
# Print the shape of the filtered arrays
for name, item in cut_nan_dict.items():
    print(f'{name} : {item.shape}')

In [ ]:
# just a check print --> If we check we do have the number of frames decreased (because NaNs where removed) --> example with worm_3
worm_name = 'worm_3'  # Change this to the name of the worm you want to print
print(f"Worm: {worm_name}")
worm_data = cut_nan_dict[worm_name]
df = pd.DataFrame(worm_data.T, columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category'])

# Check for NaN values in the DataFrame
if df.isna().sum().sum() == 0:
    print(f"Worm {worm_name} has no NaN values after NaN imputation.")
else:
    print(f"Worm {worm_name} still contains NaN values.")
df

#And we see that the total number of frames is decreased 

# Step 2 : Figure out when do the worms die
>When we find out on which frame he dies, drop the frames after his death

In [ ]:
import isdead
importlib.reload(isdead)

In [ ]:
movement_threshold = 1.0 # Threshold for inactivity detection
processed_worms = {} # Dictionary to store processed worms

dying_times = []
dying_times_frames = []

# Use the cleaned data from nan_imputation
cleaned_worms = cut_nan_dict  # Replace with the variable holding your cleaned data

# Iterate through each worm in the dataset
for worm_name, worm_data in cleaned_worms.items():
    print(f"Processing {worm_name}...")
    # Transpose worm_data for DataFrame creation
    df_worm = pd.DataFrame(worm_data.T,columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category']) # Transpose the array

    result = isdead.estimate_dying_time(df_worm, movement_threshold) # Use the estimate_dying_time function to find the dying frame
    if result[0] is None:
        print(f"  {worm_name}: No inactivity detected. Retaining all data.")
        processed_worms[worm_name] = worm_data
        continue

    dying_frame, absolute_frame, dying_time_hours, segment_number = result
  
    dying_times.append(dying_time_hours) # Append dying time in hours to the list
    dying_times_frames.append(absolute_frame) # Append the absolute frame to the list

    print(f"  {worm_name}: Dying frame = {dying_frame} of Segment = {segment_number}, Absolute frame = {absolute_frame}, Dying time = {dying_time_hours:.2f} hours") # Print details

    # Truncate the data up to the dying frame
    truncated_data = worm_data[:, worm_data[0, :] <= dying_frame]
    processed_worms[worm_name] = truncated_data

# Print summary of processed worms
print("\nSummary of processed worms:")
for name, data in processed_worms.items():
    print(f"{name}: Original frames = {worms[name].shape[1]}, After truncation = {data.shape[1]}")

In [ ]:
print(dying_times_frames)

In [ ]:
# just a check print --> Check worm 3
worm_name = 'worm_3'  # Change this to the name of the worm you want to print
print(f"Worm: {worm_name}")
worm_data = processed_worms[worm_name]
df = pd.DataFrame(worm_data.T, columns=['Frame', 'Speed', 'X', 'Y', 'Changed Pixels', 'Category'])
df

# this for a movement threshold of 1.0
# check worm 3 : Loading Data = 64794 --> Removing NaNs = 64533 frames --> Removing dead franes = 62175 Frames

In [ ]:
# Plot the survival curve
dying_times_sorted = sorted(dying_times) # Sort the dying times in ascending order

# Compute the survival rate
survival_rate = [1 - (i / len(dying_times_sorted)) for i in range(len(dying_times_sorted))]

# Plot the survival curve
plt.figure(figsize=(8, 5))
plt.plot(dying_times_sorted, survival_rate, marker='o', linestyle='-', color='blue')
plt.xlabel('Dying Time (Hours)')
plt.ylabel('Survival Rate')
plt.title('Survival Curve')
plt.grid()
plt.tight_layout()
plt.show()

# Step : Augment the Data, going from 36 to 72 worms and loading early Lifespan (35 000) frames

In [113]:
from preprocessing_features import create_aug

In [114]:
y_reg = np.array(dying_times_frames)

In [ ]:
samples = []
new_death = []
y_reg_fin = []
seed1 = 2345
seed2 = 58934823
c = 0
for name, item in processed_worms.items():
    #print(name)
    #print(item.T[0,1:5])
    arr = item.T[:,1:5]
    f1 = arr[:,0]
    f2 = arr[:,3]
    x = arr[:,1]
    y = arr[:,2]
    #create rotated sample
    rot90arr = np.vstack((f1,y,x,f2))
    x_opp, y_opp = create_aug(x,y,seed=seed1)
    x_opp2, y_opp2 = create_aug(x,y,seed=seed2)
    seed1 += 100
    seed2 -= seed1
    augarr = np.vstack((f1,x_opp,y_opp,f2))
    augarr2 = np.vstack((f1,x_opp2, y_opp2, f2))
    samples.append(arr[:35000])
    #samples.append(rot90arr.T[:35000])
    samples.append(augarr.T[:35000])
    #samples.append(augarr2.T[:35000])
    frame = item.T[:,0:1]
    #print(frame.shape)
    #print(augarr.T.shape)
    dfa = pd.DataFrame(np.hstack((frame, augarr.T)))
    dfa.columns = ['Frame','Speed','X','Y','Changed Pixels']
    y_reg_fin.append(y_reg[c])
    #y_reg_fin.append(y_reg[c])
    dying_frame,absolute_frame,dying_time_hours,segment_number = isdead.estimate_dying_time(dfa, movement_threshold=1.0)
    if absolute_frame is not None:
        new_death.append(absolute_frame)
        y_reg_fin.append(absolute_frame)
    else:
        new_death.append(arr.shape[0])
        y_reg_fin.append(arr.shape[0])
        print('ddddd')


    #second augmented
    """
    dfa2 = pd.DataFrame(np.hstack((frame, augarr2.T)))
    dfa2.columns = ['Frame','Speed','X','Y','Changed Pixels']
    dying_frame2,absolute_frame2,dying_time_hours2,segment_number2 = estimate_dying_time(dfa2, movement_threshold=1.0)
    if absolute_frame2 is not None:
        new_death.append(absolute_frame2)
        y_reg_fin.append(absolute_frame2)
    else:
        new_death.append(arr.shape[0])
        y_reg_fin.append(arr.shape[0])
        print('qqqqqq')
    """
    c += 1

In [ ]:
y_reg

In [ ]:
y_reg_fin = np.array(y_reg_fin)
y_reg_fin

In [ ]:
samples[1].shape

In [ ]:
np.min(dying_times_frames)

More imports

In [128]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
from keras.layers import AveragePooling1D
from keras.models import Model
#from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

# Step : Features extraction. Calls the function preprocess_dataset(samples) to process the data and extract features.

In [129]:
from preprocessing_features import preprocess_dataset

In [ ]:
# Process the dataset
features = list(preprocess_dataset(samples))

# Convert to DataFrame for easier analysis
features_df = pd.DataFrame(features[0:54])

print(features_df.head())

In [ ]:
death_times = []
for lifespan in processed_worms.items():
    print(f'worm name: {name}')
    arrdf = pd.DataFrame(item.T)
    if arrdf.shape[1] == 5:
        arrdf.columns = ['Frame','Speed','X','Y','Changed Pixels']
    else:
        arrdf.columns = ['Frame','Speed','X','Y','Changed Pixels','ATR']
    isdead.estimate_dying_time(arrdf, movement_threshold=1.0)
    dying_frame,absolute_frame,dying_time_hours,segment_number = isdead.estimate_dying_time(arrdf, movement_threshold=1.0)
    if absolute_frame is not None:
        death_times.append(absolute_frame)
    else:
        death_times.append(arrdf.shape[0])

In [ ]:
print(len(features))
print (y_reg) # shape of (36, )
print(y_reg_fin) # shape of (72, )

In [136]:
X = pd.DataFrame(features[0:73]) # convert to DataFrame shape will be (36, nbr of features)

In [137]:
y_reg_f =  np.array([val for val in y_reg for _ in (0, 1)]) #duplicate each element

# Step : Split the data

In [138]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_reg_fin, test_size=0.3, random_state=42)

we have 25 worms in train and 11 in test. And we have 25 features.


In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# STEP : Standardize

In [141]:
from sklearn.preprocessing import StandardScaler

In [142]:
stdsc = StandardScaler()
stdsc.fit(X_train)
X_train_std = stdsc.transform(X_train)
X_test_std = stdsc.transform(X_test)

# Step : Model 1 Linear Regression, calculate MAE

In [143]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred_lr = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred_lr))

In [145]:
y_train_pred = model.predict(X_train)

In [ ]:
larr = np.array(y_pred_lr)
stat = np.full(larr.shape, True)
time, survival_prob, conf_int = kaplan_meier_estimator(
    stat, larr, conf_type="log-log"
)

larrtr = np.array(y_test)
stattr = np.full(larrtr.shape, True)
time2, survival_prob2, conf_int2 = kaplan_meier_estimator(
    stattr, larrtr, conf_type="log-log"
)

plt.step(time, survival_prob, where="post", label='model prediction')
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.step(time2, survival_prob2, where="post", label='real times')
plt.fill_between(time2, conf_int2[0], conf_int2[1], alpha=0.25, step="post")
plt.legend()
plt.ylim(0, 1)
plt.ylabel(r"est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")

# Step : Model 2 : Ridge Regression

In [147]:
from sklearn.linear_model import Ridge, Lasso

In [ ]:
model = Ridge(alpha=1.0)  # Use Lasso(alpha=1.0) for feature selection
model.fit(X_train, y_train)
y_pred_rr = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred_rr))

In [ ]:
larr = np.array(y_pred_rr)
stat = np.full(larr.shape, True)
time, survival_prob, conf_int = kaplan_meier_estimator(
    stat, larr, conf_type="log-log"
)

larrtr = np.array(y_test)
stattr = np.full(larrtr.shape, True)
time2, survival_prob2, conf_int2 = kaplan_meier_estimator(
    stattr, larrtr, conf_type="log-log"
)

plt.step(time, survival_prob, where="post", label='model prediction')
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.step(time2, survival_prob2, where="post", label='real times')
plt.fill_between(time2, conf_int2[0], conf_int2[1], alpha=0.25, step="post")
plt.legend()
plt.ylim(0, 1)
plt.ylabel(r"est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")

# Step : model = DecisionTree Regressor

In [150]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:

model = DecisionTreeRegressor(max_depth=5)
model.fit(X_train_std, y_train)
y_pred_dtr = model.predict(X_test_std)
print('MAE:', mean_absolute_error(y_test, y_pred_dtr))
len(y_pred_dtr)

In [ ]:
larr = np.array(y_pred_dtr)
stat = np.full(larr.shape, True)
time, survival_prob, conf_int = kaplan_meier_estimator(
    stat, larr, conf_type="log-log"
)

larrtr = np.array(y_test)
stattr = np.full(larrtr.shape, True)
time2, survival_prob2, conf_int2 = kaplan_meier_estimator(
    stattr, larrtr, conf_type="log-log"
)

plt.step(time, survival_prob, where="post", label='model prediction')
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.step(time2, survival_prob2, where="post", label='real times')
plt.fill_between(time2, conf_int2[0], conf_int2[1], alpha=0.25, step="post")
plt.legend()
plt.ylim(0, 1)
plt.ylabel(r"est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")

# Step : Model ; random forest

In [153]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred_rfr = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred_dtr))

In [ ]:
larr = np.array(y_pred_rfr)
stat = np.full(larr.shape, True)
time, survival_prob, conf_int = kaplan_meier_estimator(
    stat, larr, conf_type="log-log"
)

larrtr = np.array(y_test)
stattr = np.full(larrtr.shape, True)
time2, survival_prob2, conf_int2 = kaplan_meier_estimator(
    stattr, larrtr, conf_type="log-log"
)

plt.step(time, survival_prob, where="post", label='model prediction')
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.step(time2, survival_prob2, where="post", label='real times')
plt.fill_between(time2, conf_int2[0], conf_int2[1], alpha=0.25, step="post")
plt.legend()
plt.ylim(0, 1)
plt.ylabel(r"est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")

# Step : model = SVG

In [156]:
from sklearn.svm import SVR

In [ ]:
model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
model.fit(X_train, y_train)
y_pred_svm = model.predict(X_test)
print('MAE:', mean_absolute_error(y_test, y_pred_svm))

In [ ]:
larr = np.array(y_pred_svm)
stat = np.full(larr.shape, True)
time, survival_prob, conf_int = kaplan_meier_estimator(
    stat, larr, conf_type="log-log"
)

larrtr = np.array(y_test)
stattr = np.full(larrtr.shape, True)
time2, survival_prob2, conf_int2 = kaplan_meier_estimator(
    stattr, larrtr, conf_type="log-log"
)

plt.step(time, survival_prob, where="post", label='model prediction')
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.step(time2, survival_prob2, where="post", label='real times')
plt.fill_between(time2, conf_int2[0], conf_int2[1], alpha=0.25, step="post")
plt.legend()
plt.ylim(0, 1)
plt.ylabel(r"est. probability of survival $\hat{S}(t)$")
plt.xlabel("time $t$")

# Step X : Print all features Column

In [ ]:
features_df.columns